In [11]:
import pandas as pd
import numpy as np
import datetime

In [19]:
item_prices = pd.read_csv('../data/raw/item_prices.csv')
item_sales = pd.read_csv('../data/raw/item_sales.csv')
calendar = pd.read_csv('../data/raw/daily_calendar_with_events.csv')

In [ ]:
regions = item_sales.groupby('region')
all_regions_distinct = list(item_sales['region'].unique())
itens = item_sales.groupby('item')
all_itens_distinct = list(item_sales['item'].unique())
total_days = len([i for i in item_sales if i.startswith('d_')])

In [ ]:
item_prices['yearweek'] = item_prices['yearweek'].astype('Int64')
last_yearweek = item_prices['yearweek'].min()
for index, row in item_prices.iterrows():
    if pd.isnull(row['yearweek']):
        last_yearweek += 1
        print(last_yearweek)
        item_prices.iloc[index]['yearweek'] = last_yearweek
    else:
        last_yearweek = row['yearweek']

# item_prices_without_yearweek_na = item_prices
# item_prices['yearweek'] = item_prices['yearweek'].astype('Int64').astype("string")
# item_prices['yearweek_'] = pd.to_datetime(item_prices['yearweek']+'0', format='%Y%U%w')
# test = item_prices[item_prices['yearweek_'] == pd.NaT]
# item_prices['yearweek_'].isna()
# yearweek_range
# test.head(10)




In [15]:
item_sales.head(10)

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,ACCESORIES_1_006_NYC_1,ACCESORIES_1_006,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,ACCESORIES_1_007_NYC_1,ACCESORIES_1_007,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,1,0,1,0,0,1,1
7,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,12,15,0,...,0,0,1,37,3,4,6,3,2,1
8,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2,0,7,...,0,0,1,1,6,0,0,0,0,0
9,ACCESORIES_1_010_NYC_1,ACCESORIES_1_010,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,1,...,1,0,0,0,0,0,0,2,0,2


In [20]:
melted = item_sales.melt(id_vars=['item', 'category', 'department', 'store_code', 'region'], value_vars=[i for i in item_sales if i.startswith('d_')], var_name=['days'], value_name='sales')
merged = pd.merge(melted, calendar, how='inner', left_on = 'days', right_on = 'd')
merged.head(10)

,item,category,department,store_code,region,days,sales,date,weekday,weekday_int,d,event
0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
2,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
3,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
4,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
5,ACCESORIES_1_006,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
6,ACCESORIES_1_007,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
7,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,12,2011-01-29,Saturday,1,d_1,NaN
8,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,2,2011-01-29,Saturday,1,d_1,NaN
9,ACCESORIES_1_010,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN


In [21]:
merged.info()
merged['date'] = pd.to_datetime(merged['date'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58327370 entries, 0 to 58327369
Data columns (total 12 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   item         object
 1   category     object
 2   department   object
 3   store_code   object
 4   region       object
 5   days         object
 6   sales        int64 
 7   date         object
 8   weekday      object
 9   weekday_int  int64 
 10  d            object
 11  event        object
dtypes: int64(2), object(10)
memory usage: 5.6+ GB


In [ ]:
# # date = '201328'
# # r = datetime.datetime.strptime(f'{date[:4]}-W{date[4:]}' + '-1', "%Y-W%W-%w")
# # print(f'{date[:4]}-W{date[4:]}')

# prices_per_item_and_store = item_prices[(item_prices['item'] == 'ACCESORIES_1_001') & (item_prices['store_code'] == 'NYC_1')]
# prices_per_item_and_store['yearweek'] = prices_per_item_and_store['yearweek'][:4]
# # prices_per_item_and_store['start_week_date'] = pd.to_datetime(prices_per_item_and_store['yearweek'], format='%Y-W%W')
# prices_per_item_and_store
# # item_price = prices_per_item_and_store.loc[(prices_per_item_and_store['start_week_date'] >= '2020-09-01') & (prices_per_item_and_store['start_week_date'] < '2020-09-15')]
# # item_price

In [ ]:
# regions.get_group('Philadelphia')['store_code'].unique()
sales_per_item_and_region = item_sales[(item_sales['item'] == 'SUPERMARKET_3_824') & (item_sales['store_code'] == 'NYC_2')]
sales_per_item_and_region = sales_per_item_and_region[[i for i in sales_per_item_and_region if i.startswith('d_')]]
sales_per_item_and_region_transposed = sales_per_item_and_region.T.reset_index()
sales_per_item_and_region = sales_per_item_and_region_transposed[sales_per_item_and_region_transposed.columns[1]]
sales_per_item_and_region

In [ ]:
regions_data = {}
for region in all_regions_distinct:
    itens_data = {}
    stores_per_region = regions.get_group(region)['store_code'].unique()
    sales_per_region = item_sales[item_sales['region'] == region]
    for item in all_itens_distinct:
        itens_data[item] = calendar[['date', 'weekday', 'weekday_int', 'event', 'd']]
        itens_data[item]['item'] = item
        itens_data[item]['category'] = itens.get_group(item)['category'].iloc[0]
        for store in stores_per_region:
            itens_data[item][f'{store}_sale'] = np.nan
            itens_data[item][f'{store}_price'] = np.nan
            itens_data[item][f'{store}_departament'] = np.nan
    regions_data[region] = itens_data

In [ ]:
from pathlib import Path
for region in regions_data:
    Path(f"../data/processed/{region}").mkdir(parents=True, exist_ok=True)
    for item in regions_data[region]:
        df = pd.DataFrame(regions_data[region][item])
        df.to_csv(f'../data/processed/{region}/{item}.csv', index=False)

In [ ]:
# Post-Process Region Boston
itens_per_region = item_sales[item_sales['region'] == 'Boston']['item'].unique()
stores_per_region = item_sales[item_sales['region'] == 'Boston']['store_code'].unique()
for item in itens_per_region:
    df = pd.read_csv(f'../data/processed/Boston/{item}.csv', index_col='date', parse_dates=['date'], date_parser='%Y-%m-%d')
    for store in stores_per_region:
        prices_per_item_and_store = item_prices((item_prices['item'] == item) & (item_prices['store_code'] == store))
        prices_per_item_and_store['start_week_date'] = pd.to_datetime(prices_per_item_and_store['yearweek'], format='%Y%W%w')
        item_price = prices_per_item_and_store.loc[(df['date'] >= '2020-09-01')
                     & (df['date'] < '2020-09-15')]
        sales_per_item_and_store = item_sales[(item_sales['item'] == item) & (item_sales['store_code'] == store)]
        sales_per_item_and_store  = sales_per_item_and_store[[i for i in sales_per_item_and_store if i.startswith('d_')]]
        sales_per_item_and_store_transposed = sales_per_item_and_store.T.reset_index()
        df[f'{store}_sale'] = sales_per_item_and_store_transposed[sales_per_item_and_store_transposed.columns[1]]
        df[f'{store}_price'] = sales_per_item_and_store['departament'][0]
        df[f'{store}_departament'] = sales_per_item_and_region['departament'][0]
    df.to_csv(f'../data/processed/Boston/{item}.csv', index=False)

In [ ]:
regions_data = {}
for region in all_regions_distinct:
    itens_data = {}
    stores_per_region = regions.get_group(region)['store_code'].unique()
    sales_per_region = item_sales[item_sales['region'] == region]
    for item in all_itens_distinct:
        itens_data[item] = calendar[['date', 'weekday', 'weekday_int', 'event', 'd']]
        itens_data[item]['item'] = item
        itens_data[item]['category'] = itens.get_group(item)['category'].iloc[0]
        sales_per_item = sales_per_region[sales_per_region['item'] == item]
        for store in stores_per_region:
            sales_per_store = sales_per_item[sales_per_region['store_code'] == store]
            sales_per_store = sales_per_store[[i for i in sales_per_item_and_region if i.startswith('d_')]]
            itens_data[item][f'{store}_sale'] = sales_per_store.T
            itens_data[item][f'{store}_price'] = np.nan
            itens_data[item][f'{store}_departament'] = np.nan
    regions_data[region] = itens_data

In [ ]:
regions_data['Boston']['ACCESORIES_1_003']